<a href="https://colab.research.google.com/github/ryane-jon/Natural-Language-Processing-Coursework/blob/main/NLP_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Proccessing (CE4145) Coursework
- Ryan Jones (2208751)
- Dataset Source: [Sentiment Labelled Sentences](https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences)

I plan to use various NLP techniques to create a model that can predict the sentiment (Positive or Negative) of various sentences. I also plan to extend this to detect the source of these sentences (Amazon, imdb or yelp)

### Import dataset
(from the download of the dataset in the provided link)

The 3 files are imported (amazon_cells_labelled.txt, imdb_labelled.txt, yelp_labelled.txt)

In [19]:
#import stuff to use later
from sklearn.pipeline import Pipeline #let's import the pipeline functionality
from sklearn.feature_extraction.text import CountVectorizer #and we will import a simple pre-processing method
from sklearn.feature_extraction.text import TfidfTransformer #and a representation learner
from sklearn.neighbors import KNeighborsClassifier #and a simple classifier model
from sklearn.model_selection import StratifiedKFold #cross fold is sometimes called k-fold. Calling the stratified version ensures that classes have equal representation across folds
from sklearn.metrics import accuracy_score #import an accuracy metric to tell us how well the model is doing
#used to upload data
import io
from google.colab import files
uploaded = files.upload() #files can be uploaded here, I select the 3 labelled files.

Saving amazon_cells_labelled.txt to amazon_cells_labelled.txt
Saving imdb_labelled.txt to imdb_labelled.txt
Saving yelp_labelled.txt to yelp_labelled.txt


Split each files lines into strings in arrays to be processed

In [20]:
amazon_data = uploaded['amazon_cells_labelled.txt'].decode('UTF-8').splitlines()
imdb_data = uploaded['imdb_labelled.txt'].decode('UTF-8').splitlines()
yelp_data = uploaded['yelp_labelled.txt'].decode('UTF-8').splitlines()

#View some of the data
for index in range(5):
  print(amazon_data[index])
  print(imdb_data[index])
  print(yelp_data[index])

So there is no way for me to plug it in here in the US unless I go by a converter.	0
A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  	0
Wow... Loved this place.	1
Good case, Excellent value.	1
Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  	0
Crust is not good.	0
Great for the jawbone.	1
Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  	0
Not tasty and the texture was just nasty.	0
Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!	0
Very little music or anything to speak of.  	0
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.	1
The mic is great.	1
The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  	1
The selection on the menu was great

Create a numpy array with 3 collumns: Review, Sentiment, and Source

In [21]:
import numpy as np
reviews = []
sentiments = []
sources = []

for line in amazon_data:
  reviews.append(line[:-1])
  sentiments.append(line[-1])
  sources.append("Amazon")

for line in imdb_data:
  reviews.append(line[:-1])
  sentiments.append(line[-1])
  sources.append("IMDB")

for line in yelp_data:
  reviews.append(line[:-1])
  sentiments.append(line[-1])
  sources.append("Yelp")

data = np.column_stack((reviews, sentiments, sources))
print(data.shape)

(3002, 3)


Test NLP Pipeline for Sentiment prediction (based on reviews)

In [33]:
reviews, sentiments, sources = data.T

text_clf = Pipeline([ #Pipeline to organise functions
  ('prep', CountVectorizer()), #Count vectorizer
  ('rep', TfidfTransformer()), #representation learning method using tf-idf
  ('mod', KNeighborsClassifier()), #kNN classifier
  ])

acc_score = []

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = reviews[train], reviews[test], sentiments[train], sentiments[test]
  text_clf.fit(x_train, y_train) #fit to training data
  predictions = text_clf.predict(x_test) #predict on test data
  acc_score.append(accuracy_score(predictions, y_test)) #get accuracy

print("Accuracy:", np.mean(acc_score)) #mean accuracy


Accuracy: 0.7721491957848031


Test NLP Pipeline for Source prediction (based on reviews)

In [ ]:
text_clf = Pipeline([ #Pipeline to organise functions
  ('prep', CountVectorizer()), #Count vectorizer
  ('rep', TfidfTransformer()), #representation learning method using tf-idf
  ('mod', KNeighborsClassifier()), #kNN classifier
  ])

acc_score = []

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = reviews[train], reviews[test], sources[train], sources[test]
  text_clf.fit(x_train, y_train) #fit to training data
  predictions = text_clf.predict(x_test) #predict on test data
  acc_score.append(accuracy_score(predictions, y_test)) #get accuracy

print("Accuracy:", np.mean(acc_score)) #mean accuracy